In [26]:
from requests import get
from bs4 import BeautifulSoup
import xlrd 

#Location of the excel file containg hotel links,city id and stateid
loc = ("C:/Users/prudh/Desktop/INFO 6210 DMDD/Project/hotels_urls.xlsx")

#opeing the workbook from the above mentioned location
wb = xlrd.open_workbook(loc) 

#Fetching data from sheet 1
sheet = wb.sheet_by_index(0)   

In [27]:
import re

#Declaration of hotel and address variables
hotel_name=[]
street_name=[]
landmark=[]
rating = []
review_count=[]
price=[]
city_id=[]
state_id=[]
building_num=[]
hotel_id=[]
address_id=[]
count=1


#Looping over links in the excel sheet
for i in range(sheet.nrows): 
    url = sheet.cell_value(i,0)
    response=get(url)
    
    soup = BeautifulSoup(response.text,'html.parser')
    hotels_list=soup.find_all('section' ,class_='hotel-wrap')
   
    #Looping over the response from received from each link
    for hotel_data in hotels_list:
        #city id field
        city_id.append(sheet.cell_value(i,1))
        
        #State id field
        state_id.append(sheet.cell_value(i,2))
        
        #autogeneration of hotel_id
        hotelid='htl'+str(count)
        hotel_id.append(hotelid)
        
        #Autogeneration of address id
        addid='adr'+str(count)
        address_id.append(addid)
        count=count+1
        
        #Retrieving the hotel name
        name=hotel_data.find("a",class_="property-name-link").text
        hotel_name.append(name)
    
        #Retrieving the complete address of each hotel
        if (hotel_data.find("span",dir="ltr")) is not None:
            add = hotel_data.find("span",dir="ltr").text
            addr=add.split(', ')
            #Retriving the building number form each address string
            if re.search(r'\d+', addr[0]) is not None:
                    buildingno=re.search(r'\d+', addr[0]).group(0)
                    building_num.append(buildingno)
                    #Retriving Street Name form each address String
                    street=''.join(c if c not in map(str,range(0,10)) else "" for c in addr[0])
                    street_name.append(street)
    
                    #Retriving landmark of each hotel from the response
                    land=hotel_data.find("li").text
                    landmark.append(land)
            else:
                    building_num.append('n/a')  
                    street_name.append('n\a')
                    landmark.append('n\a')
                   
        else:
           addr=""
           building_num.append('n\a')  
           street_name.append('n\a')
           landmark.append('n\a')
        
       
        
        #Retriving rating of each hotel from the response
        if hotel_data.find("div",class_="ta-logo") is not None:
            rat=hotel_data.find("div",class_="ta-logo").text
            rat=rat[-3:]
            rat=rat.replace(':','')
            rating.append(rat)
        else:
            rating.append("no rating available")
        
        #Retriving review count for each hotel from the response
        if hotel_data.find("div", class_="ta-total-reviews") is not None:
            revcnt=hotel_data.find("div", class_="ta-total-reviews").text
            review_count.append(revcnt)
        else:
            review_count.append("No review available")
        
        #Retriving price per night for each hotel from the response
        pr=hotel_data.find('ins')
        pr=str(pr)
        pr=re.findall('\d+',pr)
        p = ''.join(str(e) for e in pr)
        price.append(p)

In [28]:
import pandas as ps

#Appending all the data of hotels to a data frame
hotels  = ps.DataFrame({"Hotel Id":hotel_id,
                        "HOTEL_NAME":hotel_name,
                        "LANDMARK":landmark,
                        "RATING":rating,
                        "TOTAL REVIEWERS":review_count,
                        "PRICE PER NIGHT":price,
                        "Address Id":address_id})

In [29]:
#Displaying hotels data frame
hotels

,Hotel Id,HOTEL_NAME,LANDMARK,RATING,TOTAL REVIEWERS,PRICE PER NIGHT,Address Id
0,htl1,AC Hotel by Marriott Boston Cleveland Circle,4.7 miles to City center,4.5,83 reviews,319,adr1
1,htl2,Holiday Inn Express Boston,2.1 miles to City center,4,925 reviews,189,adr2
2,htl3,Hyatt Regency Boston,n,4,"4,683 reviews",,adr3
3,htl4,Hyatt Place Boston/Braintree,9.6 miles to City center,4,"1,189 reviews",125,adr4
4,htl5,Comfort Inn & Suites Logan International Airport,4.9 miles to City center,4.5,"1,302 reviews",178,adr5
5,htl6,Seaport Hotel,1.3 miles to City center,4.5,"5,633 reviews",332,adr6
6,htl7,Revere Hotel Boston Common,0.6 miles to City center,4.5,"5,001 reviews",584,adr7
7,htl8,Boston Marriott Copley Place,1.1 miles to City center,4,"3,671 reviews",574,adr8
8,htl9,Hilton Garden Inn Boston Logan Airport,3.4 miles to City center,4.5,"1,287 reviews",229,adr9
9,htl10,Crowne Plaza JFK Airport New York City,12 miles to City center,3.5,923 reviews,231,adr10


In [30]:
len(building_num)
#len(street_name)
#len(city_id)
#len(state_id)
#len(address_id)

228

In [31]:
#Apping fiels required for address table into a data frame
address_table=ps.DataFrame({"Building number":building_num,
                            "Street Name":street_name,
                            "City Id":city_id,
                            "State id":state_id,
                           "Address Id":address_id})

In [32]:
#Display of address table data frame
address_table

,Building number,Street Name,City Id,State id,Address Id
0,395,Chestnut Hill Ave,boston,massachusetts,adr1
1,69,Boston St,boston,massachusetts,adr2
2,n/a,n,boston,massachusetts,adr3
3,50,Forbes Road,boston,massachusetts,adr4
4,85,American Legion Hwy,boston,massachusetts,adr5
5,1,Seaport Lane,boston,massachusetts,adr6
6,200,Stuart St,boston,massachusetts,adr7
7,110,Huntington Ave,boston,massachusetts,adr8
8,100,Boardman St,boston,massachusetts,adr9
9,138,- th Ave,newyorkcity,newyork,adr10


In [33]:
export_csv = hotels.to_csv(r'C:\Users\prudh\Desktop\INFO 6210 DMDD\Project\CSV files\hotel_webscr_latest.csv', index = None, header = True)

In [34]:
export_csv = address_table.to_csv(r'C:\Users\prudh\Desktop\INFO 6210 DMDD\Project\CSV files\address_latest.csv', index = None, header = True)

In [35]:
"""CONCLUSIONS:

This code uses the list of cities, selected for the project and  tries to find the popularity parameters related 
to the same through Twitter API, stores the results in dataframe and exports it to a CSV file.

CONTRIBUTIONS:

RAJENDRA KUMAR RAJKUMAR - 40% 
MONISH  HIRISAVE RAGHU - 20%
PRUDHVI CHANDRA SEKHARAMAHANTI -40%

CITATIONS:

1. https://www.geeksforgeeks.org
2. https://github.com/nikbearbrown/INFO_6210

The code with regard to extraction of information from twitter was used from the above mentioned resources

Original writtten code - 60%
Code referenced from external sources(but modified suiting needs) - 40%

LICENSE:

Copyright <2019> <RAJENDRA KUMAR RAJKUMAR, MONISH  HIRISAVE RAGHU>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.


"""

'CONCLUSIONS:\n\nThis code uses the list of cities, selected for the project and  tries to find the popularity parameters related \nto the same through Twitter API, stores the results in dataframe and exports it to a CSV file.\n\nCONTRIBUTIONS:\n\nRAJENDRA KUMAR RAJKUMAR - 40% \nMONISH  HIRISAVE RAGHU - 20%\nPRUDHVI CHANDRA SEKHARAMAHANTI -40%\n\nCITATIONS:\n\n1. https://www.geeksforgeeks.org\n2. https://github.com/nikbearbrown/INFO_6210\n\nThe code with regard to extraction of information from twitter was used from the above mentioned resources\n\nOriginal writtten code - 60%\nCode referenced from external sources(but modified suiting needs) - 40%\n\nLICENSE:\n\nCopyright <2019> <RAJENDRA KUMAR RAJKUMAR, MONISH  HIRISAVE RAGHU>\n\nPermission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merg